In [1]:
from pyspark.context import SparkContext
from pyspark import SparkConf,sql
from pyspark.sql.session import SparkSession
import findspark
import pymysql
import pymysql.cursors
import pandas as pd

In [2]:
findspark.init()
findspark.find()

'C:\\spark\\spark-3.5.0-bin-hadoop3'

In [3]:
product_path='D:/PROJECTS/FASHIONPASS/Database/product.csv'
tagcloud_path='D:/PROJECTS/FASHIONPASS/Database/tag_cloud.csv'
collection_category_tag_path='D:/PROJECTS/FASHIONPASS/Database/collection_category_tag.csv'
product_tag_path='D:/PROJECTS/FASHIONPASS/Database/product_tag.csv'
features_path='D:/PROJECTS/FASHIONPASS/Database/spars_dense_features.csv'
collection_category_path='D:/PROJECTS/FASHIONPASS/Database/collection_category.csv'

In [4]:
spark = SparkSession.builder \
    .appName("PySpark MySQL Example") \
    .config("spark.driver.extraClassPath", "C:/spark/spark-3.5.0-bin-hadoop3/jars/mysql-connector-java-5.1.46.jar") \
    .getOrCreate()



In [8]:
table='collection_category'

In [6]:
def Dataframe_Load(table_name):
    mysqlldf=spark.read \
    .format("jdbc") \
    .option("driver","com.mysql.jdbc.Driver") \
    .option("url",'jdbc:mysql://public.live-db.fashionpass.com:3306/banjo_live') \
    .option("dbtable",table) \
    .option("user",'sa') \
    .option("fetchSize", 1000000)  \
    .option('password','Forests55_') \
    .load()
    return mysqlldf

In [9]:
mysqlldf=Dataframe_Load('collection_category')

All_Collection_Layer

In [10]:
Layer_0=mysqlldf.filter(mysqlldf['parent_id']==2)
Layer_1=mysqlldf.filter(mysqlldf['id'].isin([68,69]))
Layer_2_Clothing=mysqlldf.filter(mysqlldf['parent_id'].isin([68]))
Layer_2_Accessories=mysqlldf.filter(mysqlldf['parent_id'].isin([69]))
Layer_3_Clothing=mysqlldf.filter(mysqlldf['id'].isin([70,71,72,73,74,75,76,77,78,79,80,81,82,83,785,904,1050]))
Layer_3_Accessories=mysqlldf.filter(mysqlldf['id'].isin([249,250,251,252,253,254,255,256,1021,1122]))
Layer_4_Occasion=mysqlldf.filter(mysqlldf['id'].isin([19,21,22,37]))
Layer_5_Material_Color_Pattern=mysqlldf.filter(mysqlldf['id'].isin([17,18,62]))
Layer_6_Season=mysqlldf.filter(mysqlldf['id'].isin([23]))
Layer_7_Sleeve_Style=mysqlldf.filter(mysqlldf['id'].isin([27]))

In [5]:
Product_df=spark.read.format('csv').option('header',True) \
.load(product_path)

Tagcloud_df=spark.read.format('csv').option('header',True) \
.load(tagcloud_path)


Collection_Category_tag_df=spark.read.format('csv').option('header',True) \
.load(collection_category_tag_path)

Product_tag_df=spark.read.format('csv').option('header',True) \
.load(product_tag_path)


features_df=spark.read.format('csv').option('header',True) \
.load(features_path)


Collection_Category_df=spark.read.format('csv').option('header',True)\
.load(collection_category_path)

In [6]:
def duplicate_col(data):
  df_cols = data.columns
  duplicate_col_index = [idx for idx,
  val in enumerate(df_cols) if val in df_cols[:idx]]
  for i in duplicate_col_index:
    df_cols[i] = df_cols[i] + '_duplicate_'+ str(i)
  data = data.toDF(*df_cols)
  return data

In [7]:
Collection_Category_tag_df.show()

+---+---+-----------+------+----------+----------+-------------------+-------------------+
|_c0| id|category_id|tag_id|created_by|updated_by|         created_at|         updated_at|
+---+---+-----------+------+----------+----------+-------------------+-------------------+
|  0|  1|         12|     2|         2|        28|2020-07-02 13:51:13|2020-08-06 10:33:17|
|  1|  2|         12|     3|         2|        28|2020-07-02 13:51:19|2020-08-06 10:25:52|
|  2|  3|         12|     4|         2|         6|2020-07-02 13:51:53|2020-07-13 06:46:03|
|  3|  4|         12|     5|         2|         6|2020-07-02 13:52:01|2020-07-13 06:48:32|
|  4|  5|         19|     6|         2|         2|2020-07-02 17:05:44|2020-07-02 17:08:18|
|  5|  6|         19|     7|         2|         6|2020-07-02 17:09:45|2023-12-08 08:43:47|
|  6|  7|         19|     8|         2|       623|2020-07-02 17:10:11|2023-04-19 10:48:42|
|  7|  8|         33|    10|         2|         2|2020-07-05 20:38:31|2020-07-05 20:38:49|

In [30]:
# clothing_collection = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 785, 904, 1050, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 141, 142, 690, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 229, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 1051, 1052, 1053]
#added some collection
clothing_collection = [68,61,60,59,58,57,56,55,54,53,52,50,49,48,47,46,45,42,40,39,70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 785, 904, 1050, 84, 85, 86, 87, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 141, 142, 690, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 229, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 1051, 1052, 1053]

clothing_tag=[39,40,42,45,49,52,53,54,55,56,57,58,59,60,61,1049,1128,36,83,84,99,104,105,214,218,219,220,221,222,224,226,228,246,265,320,340,238,240,202,86,245,102,251,252,253,254,256,257,266,223,267,319,230,350,382,407,620,
              56,133,81,119,82,268,255,269,112,157,348,349,355,374,375,383,384,389,397,398,404,405,409,415,417,421,622,623,624,55,625,626,627,628,629,630,631,
659,216,307,308,309,694,24,25,26,27,28,321,352,
80,
236,
326,
393,
390,
662,
663,
381,
664,
27,
28,
665,
666,
227,
379,
419,
24,
57,
58,
59,
62,
63,
65,
67,
68,
78,
20,
21,
22,
248,
244,
670,
671,
672,
673,
106,
108,
109,
115,
122,
124,
125,
126,
129,
54,
134,
137,
138,
140,
153,
156,
154,
377,
231,
423,
159,
161,
163,
164,
165,
153,
166,
112,
167,
168,
170,
173,
179,
180,
138,
392,
400,
402,
639,
183,
184,
170,
112,
161,
669,
194,
163,
195,
161,
153,
196,
47,
48,
197,
198,
164,
200,
112,
392,
392,
204,
668,
171,
163,
195,
210,
52,
180,
392,
213,
215,
217,
153,
115,
132,
229,
67,
231,
376,
54,
49,
51,
237,
227,
242,
53,
248,
153,
76,
250,
61,
69,
66,
244,
231,
103,
120,
140,
54,
227,
38,
40,
43,
46,
47,
48,
49,
50,
51,
52,
39,
377,
191,
587,
7,
8,
270,
272,
273,
76,
73,
75,
498,
607
]

In [10]:
clothing_collection_df=Collection_Category_df.filter(Collection_Category_df['id'].isin(clothing_collection))

In [11]:
collection_category_tag_cloud=clothing_collection_df.join(Collection_Category_tag_df,clothing_collection_df.id==Collection_Category_tag_df.category_id,'left')
collection_category_tag_cloud=collection_category_tag_cloud.join(Tagcloud_df,collection_category_tag_cloud.tag_id==Tagcloud_df.id,'left')

In [12]:
collection_category_tag_cloud=duplicate_col(collection_category_tag_cloud)
collection_category_tag_cloud.show()

+---+--------------------+---------+----------+-------------+--------+-----------+------+-----------------+------+------+------------+------------+----------+----------+----------+-------------------+-------------------+---+---------------+-----------+------+-----------------------+-----------------------+-----------------------+-----------------------+----------------+---------------+--------------------+-----------------------+-----------------------+-----------------------+-----------------------+
| id|                name|parent_id|sort_order|category_link|url_link|description|status|filtersets_status|seo_id|handle|minimum_tags|maximum_tags|heading_id|created_by|updated_by|         created_at|         updated_at|_c0|id_duplicate_19|category_id|tag_id|created_by_duplicate_22|updated_by_duplicate_23|created_at_duplicate_24|updated_at_duplicate_25|_c0_duplicate_26|id_duplicate_27|            tag_name|updated_at_duplicate_29|updated_by_duplicate_30|created_at_duplicate_31|created_by_du

In [13]:
collection_category_tag_cloud=collection_category_tag_cloud.select(['id','name','tag_name','tag_id'])

In [14]:
clothing_collection_category_tag_cloud=collection_category_tag_cloud.toPandas()

In [15]:
clothing_collection_category_tag_cloud.to_csv('D:/PROJECTS/FASHIONPASS/Database/cloting_collection_category_10_1_2023.csv')

In [16]:
clothing_tag=[ i[0] for i in  collection_category_tag_cloud.select('tag_id').collect()]

In [25]:
product_product_tag_df=Product_tag_df.join(Tagcloud_df,Product_tag_df.tag_id==Tagcloud_df.id,'left')

In [26]:
product_product_tag_cloting_df=product_product_tag_df.filter(product_product_tag_df['id'].isin(clothing_tag))

In [27]:
collection_category_tag_cloud.show()

+---+--------------------+--------------------+------+
| id|                name|            tag_name|tag_id|
+---+--------------------+--------------------+------+
| 39|Activewear & Loun...|  activewear-jackets|   377|
| 39|Activewear & Loun...|          activewear|    39|
| 39|Activewear & Loun...|        biker-shorts|    52|
| 39|Activewear & Loun...|             hoodies|    51|
| 39|Activewear & Loun...|             bottoms|    50|
| 39|Activewear & Loun...|         sweatshirts|    49|
| 39|Activewear & Loun...|          sweatpants|    48|
| 39|Activewear & Loun...|            leggings|    47|
| 39|Activewear & Loun...|          yoga-pants|    46|
| 39|Activewear & Loun...|                NULL|    45|
| 39|Activewear & Loun...|           sleepwear|    43|
| 39|Activewear & Loun...|                NULL|    42|
| 39|Activewear & Loun...|                NULL|    41|
| 39|Activewear & Loun...|          loungewear|    40|
| 39|Activewear & Loun...|activewear-and-lo...|    38|
| 40|     

In [28]:
product_product_tag_cloting_df=product_product_tag_cloting_df.join(collection_category_tag_cloud,collection_category_tag_cloud.tag_id==product_product_tag_cloting_df.tag_id,'left')

In [29]:
product_product_tag_cloting_df.show()

+---+----------+------+---+---+----------------+-------------------+----------+-------------------+----------+----+--------------------+----------------+------+
|_c0|product_id|tag_id|_c0| id|        tag_name|         updated_at|updated_by|         created_at|created_by|  id|                name|        tag_name|tag_id|
+---+----------+------+---+---+----------------+-------------------+----------+-------------------+----------+----+--------------------+----------------+------+
| 42|         4|   153|119|153|         cropped|2020-08-18 17:39:23|       2.0|2020-07-13 06:14:53|         0|  45|Jackets, Coats & ...|         cropped|   153|
| 42|         4|   153|119|153|         cropped|2020-08-18 17:39:23|       2.0|2020-07-13 06:14:53|         0|  49|       Jeans & Denim|         cropped|   153|
| 42|         4|   153|119|153|         cropped|2020-08-18 17:39:23|       2.0|2020-07-13 06:14:53|         0|  54|               Pants|         cropped|   153|
| 42|         4|   153|119|153|   